### Researched Quantities

- Building this model based on a 10-year rainstorm, which is 5.5 inches in 24 hours.
-Here is the link to the sheet with the data about the plants https://1drv.ms/x/c/348b84fad346e9f1/EX-eKXt_eg1HnNybvuDh2aUBIBjV-u4fKmu33ILq_tpZVw?e=BLTTLG 
- Unit conversions:
    - 1 square mile = 640 acres
    - 1 square mile = 27,878,400 square feet
    - 1 foot = 12 inches
    - 1 gallon = 0.13368 cubic feet
- According to HomeAdvisor, it costs between $50 to $250 per linear foot to insall a sewer line. I feel like since it's NYC it'll be more expensive but it's so much that they can probably get pretty good pricing so I'm just going to guess $150 per linear foot.
    https://www.homeadvisor.com/cost/plumbing/install-a-sewer-main/

In [2]:
import Pkg
Pkg.activate(@__DIR__)
Pkg.instantiate()

using JuMP
using HiGHS
using DataFrames
using GraphRecipes
using Plots
using Measures
using MarkdownTables

  Activating project at `c:\Users\arisc\OneDrive\Desktop\BEE 4750\4750project`


In [ ]:
#deriving objective function
facilities = ["26th Ward", "Bowery Bay", "Coney Island", "Hunts Point", "Jamaica", "Newtown Creek", "North River", "Oakwood Beach", "Rockaway", "Owls Head", "Wards Island", "Tallman Island", "Port Richmond", "Red Hook"]
design_capacity = [85, 150, 110, 200, 100, 310, 170, 39.9, 45, 120, 275, 80, 60, 60] #MGD
population_served = [283428, 848328, 596326, 684569, 728123, 1068012, 588772, 244918, 90474, 758007, 1061558, 410812, 198128, 192050] #
sanitary_demand = 80*(10^-6)*population_served #MGD = (gal/(day * person)) * people * 10^-6 MGD/(gal/d)
distance = [10.25, 9.55, 10, 11, 13, 5.7, 9.7, 11.25, 14.25, 4.45, 8.75, 12.6, 6.15, 3.7] #miles
drainage_area = [5907, 15203, 15087, 16664, 25313, 15656, 6030, 10779, 6259, 12947, 12056, 16860, 9665, 3200] #acres = (miles^2)/640

#i've figured out how to calculate the total volume a wwtp will be getting but I feel like the way we're doing it we're creating a boolean variable because we'd either connect to the new wwtp or not. length is more important than volume

14-element Vector{Int64}:
  5907
 15203
 15087
 16664
 25313
 15656
  6030
 10779
  6259
 12947
 12056
 16860
  9665
  3200

In [ ]:
#Setting Up Optimization
stormwater_model = Model(HiGHS.Optimizer) #initialize model object

#create variables: amount of wastewater diverted to new plant
@variable(stormwater_model, W[1:14] >= 0)

#create objective function

#constraints

In [ ]:
#optimize
optimize!(stormwater_model)
@show value.(W)